# IETF Affiliations from Attendance Records

In [1]:
import bigbang.domain as domain
import bigbang.datasets.domains as domains

from ietfdata.datatracker     import *
from ietfdata.datatracker_ext import *
import pandas as pd
import matplotlib.pyplot as plt
import dataclasses

In [2]:
org_cats = pd.read_csv("organization_categories - organization_categories.csv")

## Getting attendance records from datatracker

When attendees register for a meeting, the report their name, email address, and affiliation.

While this is noisy data (any human-entered data is!), we will use this information to associate domains with affilations. E.g. the email domain `apple.com` is associated with the company Apple.

We will also use this data to enrich our understanding of individual affiliations over time.

In [3]:
datatracker = DataTracker()

meetings = datatracker.meetings(meeting_type = datatracker.meeting_type(MeetingTypeURI('/api/v1/name/meetingtypename/ietf/')))
full_ietf_meetings = list(meetings)

INFO:ietfdata:glasgow-ietfdata/0.5.1 (cache disabled)


In [4]:
ietf_meetings = []
for meeting in full_ietf_meetings:
    meetingd = dataclasses.asdict(meeting)
    meetingd['meeting_obj'] = meeting
    meetingd['num'] = int(meeting.number)
    ietf_meetings.append(meetingd)    

In [5]:
meetings_df = pd.DataFrame.from_records(ietf_meetings)

## Individual Affiliations

In [6]:
dt = DataTrackerExt() # initialize, for all meeting registration downloads

INFO:ietfdata:glasgow-ietfdata/0.5.1 (cache disabled)


This will construct a dataframe of every attendee's registration at every specified meeting. (Downloading this data takes a while!)

In [7]:
ietf_meetings[110]['date']

datetime.datetime(2021, 7, 24, 0, 0)

In [8]:
meeting_attendees_df = pd.DataFrame()
for meeting in ietf_meetings:
    if meeting['num'] in [104,105,106,107,108,109]: # can filter here by the meetings to analyze
        registrations = dt.meeting_registrations(meeting=meeting['meeting_obj'])
        df = pd.DataFrame.from_records([dataclasses.asdict(x) for x in list(registrations)])
        df['num'] = meeting['num']
        df['date'] = meeting['date']
        df['domain'] = df['email'].apply(domain.extract_domain)
        full_name = df['first_name'] + " " + df['last_name']
        df['full_name'] = full_name
        meeting_attendees_df = meeting_attendees_df.append(df)

Filter by those who actually attended the meeting (checked in, didn't just register).

In [9]:
ind_affiliation = meeting_attendees_df[['full_name', 'affiliation', 'email', 'domain','date']]

This format of data -- with name, email, affiliation, and a timestamp -- can also be extracted from other IETF data, such as the RFC submission metadata. Later, we will use data of this form to infer _duration_ of affilation for IETF attendees.

In [10]:
ind_affiliation[:10]

,full_name,affiliation,email,domain,date
0,Thomas Pauly,Apple,tpauly@apple.com,apple.com,2019-03-23
1,Eric Kinnear,Apple,ekinnear@apple.com,apple.com,2019-03-23
2,Jordi Palet Martinez,Moremar,jordi.palet@consulintel.es,consulintel.es,2019-03-23
3,Heather Flanagan,RFC Editor,rse@rfc-editor.org,rfc-editor.org,2019-03-23
4,Kyle Rose,Akamai Technologies,krose@krose.org,krose.org,2019-03-23
5,Aaron Falk,Akamai,aaron.falk@gmail.com,gmail.com,2019-03-23
6,Russ Housley,"Vigil Security, LLC",housley@vigilsec.com,vigilsec.com,2019-03-23
7,Jason Livingood,Comcast // IASA 2.0 WG,Jason_Livingood@comcast.com,comcast.com,2019-03-23
8,Jeff Osborn,Internet Systems Consortium,jeff@isc.org,isc.org,2019-03-23
9,Mahesh Jethanandani,VMware,mjethanandani@gmail.com,gmail.com,2019-03-23


## Matching affiliations with domains

In [11]:
affil_domain = ind_affiliation[['affiliation', 'domain', 'email']].pivot_table(
    index='affiliation',columns='domain', values='email', aggfunc = 'count')

Drop both known generic and known personal email domains.

In [12]:
ddf = domains.load_data()

generics = ddf[ddf['category'] == 'generic'].index
personals = ddf[ddf['category'] == 'personal'].index

In [13]:
generic_email_domains = set(affil_domain.columns).intersection(generics)
affil_domain.drop(generic_email_domains, axis = 1, inplace = True)

In [14]:
personal_email_domains = set(affil_domain.columns).intersection(personals)
affil_domain.drop(personal_email_domains, axis = 1, inplace = True)

In [15]:
ad_max = affil_domain.apply(lambda row: row.max(), axis=1)
ad_mean = affil_domain.apply(lambda row: row.dropna().mean(), axis=1)
ad_count = affil_domain.apply(lambda row: row.dropna().count(), axis=1)
ad_sum = affil_domain.apply(lambda row: row.dropna().sum(), axis=1)

ad_max_domain = affil_domain.apply(lambda row: row.idxmax(), axis=1)

## Add the columns *after* computing the statistics!
affil_domain['max'] = ad_max
affil_domain['mean'] = ad_mean
affil_domain['count'] = ad_count
affil_domain['sum'] = ad_sum
affil_domain['max_domain'] = ad_max_domain

In [16]:
ad_stats = affil_domain[['max_domain','max','count','mean','sum']].sort_values('max', ascending=False)

In [17]:
ad_stats[:100]

domain,max_domain,max,count,mean,sum
affiliation,,,,,
Huawei,huawei.com,132.0,4,33.750000,135.0
Cisco,cisco.com,126.0,6,23.166667,139.0
Cisco Systems,cisco.com,124.0,3,42.666667,128.0
Google,google.com,112.0,9,14.444444,130.0
Ericsson,ericsson.com,103.0,8,15.625000,125.0
...,...,...,...,...,...
Telefonica,telefonica.com,7.0,1,7.000000,7.0
CENTR,centr.org,7.0,1,7.000000,7.0
IANA Services,iana.org,7.0,1,7.000000,7.0


In [18]:
ad_stats[:100].to_csv("affiliation_domain_stats.csv")

## Duration of affiliation

The current data we have for individual affiliations is "point" data, reflecting the affiliation of an individual on a particular date.

For many kinds of analysis, we may want to understand the full duration for which an individual has been associated with an organization. This requires an inference from the available data points to dates that are not explicitly represented in the data.

For now, we will use a rather simple form of inference: filling in any missing data from the last (temporally) known data point. And then if there's still missing data, infer backwards.

In [19]:
affil_dates = ind_affiliation.pivot_table(
    index="date",
    columns="full_name",
    values="affiliation",
    aggfunc="first"
).fillna(method='ffill').fillna(method='bfill')

In [20]:
top_attendees = ind_affiliation.groupby('full_name')['date'].count().sort_values(ascending=False)[:40].index

In [21]:
top_attendees

Index(['Ignas Bagdonas', 'Martin Duke', 'Gert Grammel', 'Roni Even',
       'Linda Dunbar', 'Toerless Eckert', 'Richard Barnes', 'Kohei Isobe',
       'Yutaka OIWA', 'Jonathan Lennox', 'Jim Reid', 'Ronald in 't Velt',
       'Gonzalo Camarillo', 'Paul Ebersman', 'Martin Thomson',
       'Marten Seemann', 'Glenn Deen', 'Martin Vigoureux', 'Paul Congdon',
       'Tianran Zhou', 'Ramesh Sivakolundu', 'Tero Kivinen', 'Markus Amend',
       'Ted Hardie', 'Chris Bowers', 'Tal Mizrahi', 'Takuya Miyasaka',
       'Chonggang Wang', 'Dominique Lazanski', 'Gorry Fairhurst',
       'Dino Farinacci', 'Tadahiko Ito', 'Suzanne Woolf', 'Susan Hares',
       'Suresh Krishnan', 'Matthew Ford', 'Dieter Sibold', 'Mark Nottingham',
       'Paul Hoffman', 'Marcus Ihlar'],
      dtype='object', name='full_name')

In [22]:
affil_dates[top_attendees]

full_name,Ignas Bagdonas,Martin Duke,Gert Grammel,Roni Even,Linda Dunbar,Toerless Eckert,Richard Barnes,Kohei Isobe,Yutaka OIWA,Jonathan Lennox,...,Dino Farinacci,Tadahiko Ito,Suzanne Woolf,Susan Hares,Suresh Krishnan,Matthew Ford,Dieter Sibold,Mark Nottingham,Paul Hoffman,Marcus Ihlar
date,,,,,,,,,,,,,,,,,,,,,
2019-03-23,Equinix,"F5 Networks, Inc.",Juniper,Huawei Technologies,Futurewei,Huawei USA,Cisco,SECOM,,Vidyo,...,,,,,Kaloom,Internet Society,PTB,,ICANN,Ericsson
2019-07-20,Equinix,"F5 Networks, Inc.",Juniper Networks,Toga Networks,Futurewei,Futurewei Technologies USA,Cisco,SECOM,AIST Japan / 産業技術総合研究所,8x8,...,lispers.net,SECOM,Public Interest Registry (.org),,Kaloom,Internet Society (ISOC),PTB,Fastly,ICANN,Ericsson
2019-11-16,Equinix,"F5 Networks, Inc.",Juniper,Toga Networks,Futurewei,Futurewei USA,Cisco,SECOM,AIST Japan,8x8 / Jitsi,...,,SECOM,Public Interest Registry (.org),,Kaloom,Internet Society,PTB,Fastly,ICANN,Ericsson
2020-03-21,Equinix,F5 Networks,Juniper,Toga Networks,,Futurewei USA,Cisco,SECOM,AIST Japan,8x8 / Jitsi,...,lispers.net,,Public Interest Registry (.org),,Kaloom,Internet Society,,Fastly,ICANN,Ericsson
2020-07-25,Equinix,"F5 Networks, Inc.",Juniper,,Futurewei,Futurewei USA,Cisco,SECOM,AIST Japan,,...,lispers.net,"SECOM CO., LTD.",Public Interest Registry (.ORG),Huawei,Kaloom,Internet Society,PTB,Fastly,ICANN,Ericsson
2020-11-14,Equinix,"F5 Networks, Inc.",Juniper,,Futurewei,Futurewei USA,Cisco,SECOM,AIST Japan,8x8 / Jitsi,...,lispers.net,SECOM,Public Interest Registry (PIR),Hickory Hill Consulting,Kaloom,Internet Society (ISOC),PTB,Fastly,ICANN,Ericsson


In [23]:
affil_dates[top_attendees].to_csv("inferred_affiliation_dates.csv")

### Linking to Organization lists

In [24]:
import bigbang.process as process

In [25]:
# drop subsidiary organizations
org_cats = org_cats[org_cats['subsidiary of / alias of'].isna()]

org_cats

,name,category,subsidiary of / alias of,stakeholdergroup,nationality,email domain names,Membership Organization,Wiki Page
0,3GPP,Standards Body,NaN,Governance body,NaN,3gpp.org,NaN,NaN
1,450connect GmbH,NaN,NaN,NaN,Germany,NaN,ETSI,NaN
2,4G Americas,NaN,NaN,NaN,United States,NaN,OTHER,NaN
3,5G Americas,NaN,NaN,NaN,United States,NaN,OTHER,NaN
4,5G Automotive Association,Association,NaN,NaN,Germany,NaN,OTHER,NaN
...,...,...,...,...,...,...,...,...
902,Zhejiang University,Research Institution,NaN,Academia,China,NaN,CCSA,NaN
903,Zollkriminalamt (ZKA),NaN,NaN,NaN,Germany,NaN,ETSI,NaN
904,ZONSON SAMRT AUTO CORPORATION,NaN,NaN,NaN,China,NaN,CCSA,NaN
905,ZTE,Networking equipment vendor,NaN,Business,China,"zte.com.cn , zte.com",CCSA,NaN


Normalize/resolve the names from the IETF attedence records.

In [26]:
org_names = ad_stats['sum']
org_names = org_names.append(
    pd.Series(index = org_cats['name'], data = 1)
)
org_names = org_names.sort_values(ascending = False)
org_names = org_names[~org_names.index.duplicated(keep="first")]

In [27]:
ents = process.resolve_entities(
    org_names,
    process.containment_distance,
    threshold=.15
)

replacements = {}
for r in [{name: ent for name in ents[ent]} for ent in ents]:
    replacements.update(r)

In [28]:
ad_stats['norm_org'] = ad_stats.apply(lambda x : replacements[x.name], axis = 1)
org_cats['norm_org'] = org_cats.apply(lambda x : replacements[x['name']], axis = 1)

In [29]:
org_cats_plus = org_cats.join(ad_stats[['max_domain', 'norm_org']], on = 'norm_org', rsuffix="_ietf")

In [30]:
org_cats_plus_match = org_cats_plus[(~org_cats_plus['max_domain'].isna())].drop('norm_org_ietf',axis=1).rename({'max_domain' : 'max_domain_ietf'}, axis = 1)

In [31]:
org_cats_plus_match.to_csv("org_categories_matched_with_ietf_attendence_domains.csv")

In [32]:
org_cats_plus_match[:20]

,name,category,subsidiary of / alias of,stakeholdergroup,nationality,email domain names,Membership Organization,Wiki Page,norm_org,max_domain_ietf
23,Afilias,Internet Registry,NaN,Business,United States,NaN,NaN,NaN,Afilias,afilias.info
26,Akamai Technologies,Content Distribution Network,NaN,Business,United States,akamai.com,NaN,NaN,Akamai Technologies,akamai.com
29,Alibaba (China) Group. Ltd.,NaN,NaN,Business,China,alibaba-inc.com,CCSA,NaN,Alibaba,alibaba-inc.com
36,Amazon Web Services Inc.,Cloud Provider,NaN,Business,United States,NaN,ETSI,NaN,Amazon Web Services,amazon.com
38,AMS,IETF secretariat,NaN,Governance body,United States,NaN,NaN,NaN,AMS,amsl.com
44,APNIC,Internet Registry,NaN,Governance body,NaN,NaN,NaN,NaN,APNIC,apnic.net
45,Apple,Consumer hardware and software vendor,NaN,Business,United States,apple.com,NaN,NaN,Apple,apple.com
62,Arista Networks,Cloud Provider,NaN,Business,United States,arista.com,NaN,NaN,Arista Networks,arista.com
63,Arrcus,Cloud Provider,NaN,Business,United States,arrcus.com,NaN,NaN,Arrcus,arrcus.com
64,ARTICLE19,Civil Society Organization,NaN,Civil Society,United Kingdom,article19.org,ETSI,NaN,Article19,article19.org


## Export the graph of relations

Getting the affiliation data relations extracted from the attendance tables.

Final form: Three tables:
 - Name - Email, earliest and latest date
 - Name - Affiliation, earliest and latest date
 - Email - Affiliation, earliest and latest date

These can be combined into a tripartite graph, which should have a component for each affiliation entity.


In [50]:
meeting_range = [106,107,108]

In [71]:
def name_email_affil_relations_from_IETF_attendance(meeting_range):
    """
    
    Extract and infer from IETF attendance records relations between full names, email address, and affiliations.
    
    In the returned dataframes, each row represents a relation between two of these forms of entity,
    along with the maximum and minimum date associated with it in the data.
    
    Two forms of inference are used when generating these relational tables:
    
     - Missing values in time are filled forward, then filled backward
     - TODO: Affiliations are ran through the entity resolution script to reduce them to a 'cannonical form'
    
    Parameters
    ------------
    
    meeting_range: list of ints
        The numbers of the IETF meetings to use for source data
        
        
    Returns
    -----------
    
    rel_name_affil: pandas.DataFrame
    
    rel_email_affil: pandas.DataFrame
    
    rel_name_email: pandas.DataFrame
    
    
    """
    dt = DataTrackerExt() # initialize, for all meeting registration downloads
    
    meeting_attendees_df = pd.DataFrame()
    for meeting in ietf_meetings:
        if meeting['num'] in meeting_range: # can filter here by the meetings to analyze
            registrations = dt.meeting_registrations(meeting=meeting['meeting_obj'])
            df = pd.DataFrame.from_records([dataclasses.asdict(x) for x in list(registrations)])
            df['num'] = meeting['num']
            df['date'] = meeting['date']
            df['domain'] = df['email'].apply(domain.extract_domain)
            full_name = df['first_name'] + " " + df['last_name']
            df['full_name'] = full_name
            meeting_attendees_df = meeting_attendees_df.append(df)
            
    ind_affiliation = meeting_attendees_df[['full_name', 'affiliation', 'email', 'domain','date']]
    
    # full_name / affiliation
    name_affil_dates = ind_affiliation.pivot_table(
        index="date",
        columns="full_name",
        values="affiliation",
        aggfunc="first"
    ).fillna(method='ffill').fillna(method='bfill')

    long_name_affils = name_affil_dates.reset_index().melt(id_vars=['date'], value_name="affiliation")

    rel_name_affil = long_name_affils.groupby(['full_name', 'affiliation'])['date'] \
           .agg(['min','max']).reset_index() \
           .rename({'min' : 'min_date', 'max' : 'max_date'}, \
                   axis=1)
    
    # email / affiliation
    email_affil_dates = ind_affiliation.pivot_table(
        index="date",
        columns="email",
        values="affiliation",
        aggfunc="first"
    ).fillna(method='ffill').fillna(method='bfill')

    long_email_affils = email_affil_dates.reset_index().melt(id_vars=['date'], value_name="affiliation")

    rel_email_affil = long_email_affils.groupby(['email', 'affiliation'])['date'] \
           .agg(['min','max']).reset_index() \
           .rename({'min' : 'min_date', 'max' : 'max_date'}, \
                   axis=1)
    
    # name / email
    name_email_dates = ind_affiliation.pivot_table(
        index="date",
        columns="full_name",
        values="email",
        aggfunc="first"
    ).fillna(method='ffill').fillna(method='bfill')

    long_name_email = name_email_dates.reset_index().melt(id_vars=['date'], value_name="email")

    rel_name_email = long_name_email.groupby(['full_name', 'email'])['date'] \
           .agg(['min','max']).reset_index() \
           .rename({'min' : 'min_date', 'max' : 'max_date'}, \
                   axis=1)
    
    return rel_name_affil, rel_email_affil, rel_name_email

In [72]:
name_email_affil_relations_from_attendence(meeting_range)

INFO:ietfdata:glasgow-ietfdata/0.5.1 (cache disabled)
INFO:ietfdata:cache requests: 0
INFO:ietfdata:cache hit rate: -
INFO:ietfdata:HTTP GET calls: 38
INFO:ietfdata:db requests: 0


(                             full_name           affiliation   min_date  \
 0                                                            2019-11-16   
 1     ABDEL RAHMAN NJI PANSO MOUNCHILI                       2019-11-16   
 2                       AKIRA MURAKAMI               TOSHIBA 2019-11-16   
 3                       ALBERT ESPINAL                 ESPOL 2019-11-16   
 4                        ANDREW GROVER               Mozilla 2019-11-16   
 ...                                ...                   ...        ...   
 2726                   yusuke kagiwada                       2019-11-16   
 2727                        zhenbin Li                       2019-11-16   
 2728                       Éric Vyncke                 Cisco 2019-11-16   
 2729                Øyvind Rønningstad  Nordic Semiconductor 2019-11-16   
 2730                              承勳 蔡            恩典之前  讚美之裏 2019-11-16   
 
        max_date  
 0    2020-07-25  
 1    2020-07-25  
 2    2020-07-25  
 3    2020